# End-to-end Model training & deployment using AWS Sagemaker Platform

## 1. Import libraries & modules:

In [1]:
%cd ..
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
import os
import zipfile
import boto3
from pathlib import Path
import pandas as pd
import shutil

/home/ec2-user/SageMaker/gender-classification


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[04/26/25 15:10:11] INFO     Found credentials from IAM Role:                                   ]8;id=780696;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=241182;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## 2. Prepare data:

In [2]:
# Unzip the original dataset file
os.makedirs('data', exist_ok=True)
with zipfile.ZipFile('gender_classification_dataset.zip') as zipf:
    zipf.extractall('data')

In [3]:
# Load csv files
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [4]:
# Create folders for splitting
os.makedirs('data/train/male', exist_ok=True)
os.makedirs('data/train/female', exist_ok=True)
os.makedirs('data/test/male', exist_ok=True)
os.makedirs('data/test/female', exist_ok=True)

In [5]:
# Copy files to separate folders
for path in train_df.path:
    shutil.copy(f'data/{path}', f'data/train/{path}')
for path in test_df.path:
    shutil.copy(f'data/{path}', f'data/test/{path}')
shutil.copy('data/train.csv', 'data/train/train.csv')
shutil.copy('data/test.csv', 'data/test/test.csv')

'data/test/test.csv'

In [6]:
# Pack to zip files
with zipfile.ZipFile('train.zip', 'w') as train_zip:
    for path in train_df.path:
        train_zip.write(f'data/train/{path}', arcname=path)
    train_zip.write('data/train/train.csv', arcname='train.csv')

with zipfile.ZipFile('test.zip', 'w') as test_zip:
    for path in test_df.path:
        test_zip.write(f'data/test/{path}', arcname=path)
    test_zip.write('data/test/test.csv', arcname='test.csv')

In [7]:
# Upload to S3
bucket = 'hieunt-sagemaker-gender-classification'
prefix = 'data'
s3 = boto3.client('s3')
s3.upload_file('train.zip', bucket, f'{prefix}/train.zip')
s3.upload_file('test.zip', bucket, f'{prefix}/test.zip')

[04/26/25 08:38:04] INFO     Found credentials from IAM Role:                                   ]8;id=315986;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=203462;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [8]:
# Clean-up
shutil.rmtree('data')
Path('train.zip').unlink()
Path('test.zip').unlink()

## 3. Training the model with training script (`training.py`):

In [8]:
role = get_execution_role()

model = PyTorch(
    entry_point='train_and_deploy.py',
    source_dir='code',
    role=role,
    instance_type='ml.m5.xlarge',
    instance_count=1,
    framework_version='2.2.0',
    py_version='py310',
    hyperparameters=dict(epochs=10, batch_size=32, learning_rate=1e-3, model_filename='gender_classifier_250426.pt')
)

model.fit({'train': 's3://hieunt-sagemaker-gender-classification/data/train.zip',
           'test': 's3://hieunt-sagemaker-gender-classification/data/test.zip'})

[04/26/25 15:46:35] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=164480;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=686049;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=629460;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=527395;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=26044;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=125103;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating training-job with name:                                       ]8;id=42540;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=194632;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             pytorch-training-2025-04-26-15-46-35-502                                              

2025-04-26 15:46:39 Starting - Starting the training job...
2025-04-26 15:47:10 Downloading - Downloading input data...
2025-04-26 15:47:30 Downloading - Downloading the training image......
2025-04-26 15:48:40 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,
2025

## 4. Deploy the trained model:

In [11]:
model.deploy(instance_type='ml.m5.xlarge', initial_instance_count=1, endpoint_name='gender-classifier-v1-1',
             model_name='gender-classification-v1-1', entry_point='src/inference.py')

[04/26/25 16:14:44] INFO     Repacking model artifact                                                  ]8;id=930696;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/model.py\model.py]8;;\:]8;id=611844;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/model.py#820\820]8;;\
                             (s3://sagemaker-ap-southeast-1-406829288865/pytorch-training-2025-04-26-1             
                             5-46-35-502/output/model.tar.gz), script artifact                                     
                             (s3://sagemaker-ap-southeast-1-406829288865/pytorch-training-2025-04-26-1             
                             5-46-35-502/source/sourcedir.tar.gz), and dependencies ([]) into single               
                             tar.gz file located at                                                                
                             s3://sagemaker-ap-southeast-1-406829288865/gender-classification-v1-1/mod             
                             el.tar.gz. This may take some time depending on model size...                         

[04/26/25 16:15:02] INFO     Creating model with name: gender-classification-v1-1                   ]8;id=845073;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=284195;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

                    WARNING  Using already existing model: gender-classification-v1-1               ]8;id=460691;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=22100;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4105\4105]8;;\

                    INFO     Creating endpoint-config with name gender-classifier-v1-1              ]8;id=599499;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=728163;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\

                    INFO     Creating endpoint with name gender-classifier-v1-1                     ]8;id=554573;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=328661;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\

--------!